# Calling API to retrieve JSONs for exchange rate on historical dates

In [1]:
import os
import pandas as pd
from zipfile import ZipFile
import datetime
import requests

In [4]:
# setting up csv to be able to retrieve information
import os
import pandas as pd
from zipfile import ZipFile

zf = ZipFile('archive.zip', 'r')
zf.extract('wfp_food_prices_database.csv')
zf.close()

df = pd.read_csv('wfp_food_prices_database.csv')
os.remove('wfp_food_prices_database.csv')
df.head(5)

C:\Users\Lilly\AppData\Local\Temp\ipykernel_32780\2143704076.py:10: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('wfp_food_prices_database.csv')


,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN
1,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN
2,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN
3,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN
4,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN


In [5]:
# Create Datetime object for each row
# Isolate just the month and date of each row
tempDayMonthDF = df.loc[:, 'mp_month':'mp_year']

# Define a temp method for creating a datetime object from a row of our dataframe
def _create_datetime(row):
    return datetime.date(row[1], row[0], 1)

# Add new datetime row to the dataframe
tempDayMonthDF = tempDayMonthDF.apply(_create_datetime, axis=1)
df.insert(0, "Date", tempDayMonthDF)
df.head()

,Date,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,2014-01-01,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN
1,2014-02-01,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN
2,2014-03-01,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN
3,2014-04-01,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN
4,2014-05-01,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN


In [16]:
# retrieve unique currencies from database
symbols = list(df['cur_name'].unique())
print('USD' in symbols) #confirming to use as base
symbolsString = ', '.join(symbols)


True


In [19]:
# retrieve all unique dates in or after 1999 from database
uniqueDates = list(df['Date'].unique())
uniqueDates
past_1999Dates = list(filter(lambda x: x.year >= 1999, uniqueDates))
past_1999Dates
min(past_1999Dates)



datetime.date(1999, 1, 1)

In [22]:
#testing format of converting date to string
tempDate = min(past_1999Dates)
tempDate.strftime('%Y-%m-%d')

/


In [ ]:
# setting up API calling parameters
url = 'https://api.exchangeratesapi.io/v1/'
key = ''
base = 'USD'
# **note**: exchangerates api only allows for 250 calls per free account, therefore we need to use multiple keys to make 260 calls

responsesDict = dict()

for date in uniqueDates:
    dateString = date.strftime('%Y-%m-%d')
    requestString ="{url}/{date}?access_key={key}&base=USD&symbols={symbols}",format(url=url,date=date,key=key,symbols=symbolsString)
    responseJson = requests.get(requestString)
    responsesDict[date]=responseJson

# Changing Retrieved Jsons into Dictionary/CSV

In [ ]:
"""
Exchange_Rates is a dictionary of all historical exchange rates to USD. Each key is a date and each value is a JSON following the format below:
{
    "success": true,
    "timestamp": 1519296206,
    "base": "EUR",
    "date": "2021-03-17",
    "rates": {
        "AUD": 1.566015,
        "CAD": 1.560132,
        "CHF": 1.154727,
        "CNY": 7.827874,
        "GBP": 0.882047,
        "JPY": 132.360679,
        "USD": 1.23396,
    [...]
    }
}
(excerpt from https://exchangeratesapi.io/documentation/)

"""
Exchange_Rates = {}

# refer to a data structure from foodPriceAnalysis.ipynb with every date we must query and every currency
%store -r structure

for date in structure:
    Exchange_Rates[date] = 'API CAll result'

# https://api.exchangeratesapi.io/v1/2013-12-24
#     ? access_key = API_KEY
#     & base = GBP
#     & symbols = USD,CAD,EUR

# hardcode currencies

# populate dict and then dump in the following block

In [ ]:
"""
We'll save the exchange rates JSON permanently as the values never change and we don't need to call the API every time we wish to use any of the exchange rate information.
"""
with open('exchange_rates.json', 'w') as f:
    json.dump(Exchange_Rates, f)

"""
The exchange rates json can now be loaded into a larger object elsewhere in other notebooks such as foodPriceAnalysis.ipynb
"""